In [9]:
import pandas as pd
import pickle, os, re


def collect(df_dict):
    mode_dict = {'0':'interday', '1':'intraday'}
    act_dict = {'r':'relu', 't':'tanh', 's':'sigmoid'}

    hy_params = os.listdir('logs/')
    for hy_param in hy_params:
        hypers = hy_param.split('_')
        if len(hypers) < 4:
            continue

        act_hy_params = os.listdir('logs/'+hy_param+'/')
        for act_hy_param in act_hy_params:
            if len(act_hy_param.split('.')) > 1:
                continue

            items = os.listdir('logs/' + hy_param + '/' + act_hy_param + '/')
            for item in items:
                if item.endswith('.pkl'):
                    model_name = item.split('_')[0]
                    with open('logs/' + hy_param + '/' + act_hy_param + '/' + item, 'rb') as file:
                        history, test_loss, test_metrics = pickle.load(file)
                    history_df = pd.DataFrame(history)
                    insert = [int(hypers[0]), mode_dict[hypers[1]], hypers[2], hypers[3], act_dict[act_hy_param],
                              history_df['IC'].values[-1], history_df['val_IC'].values[-1], test_metrics]
                    df_dict[model_name].loc[len(df_dict[model_name])] = insert
    return df_dict

In [25]:
df_dict = {}
models = ['x', 'y', 'cnn', 'lstm', 'bilstm', 'tcn']
for model in models:
    df_dict[model] = pd.DataFrame(columns=['size','mode','bar','market','activation','train','valid','test'])
result = collect(df_dict)
for model, df in df_dict.items():
    df = df.sort_values(by=['market', 'mode', 'size'], ascending=True)
    df.index = range(len(df))
    df_dict[model] = df



In [26]:
for model, df in df_dict.items():
    print(model)
    print(df)

x
  size      mode bar market activation     train     valid      test
0    1  interday  15      e       relu  0.102164  0.101887  0.123767
1    2  interday  15      e       relu  0.111260  0.098029  0.125080
2    3  interday  15      e       relu  0.109132  0.099208  0.120673
3    4  interday  15      e       relu  0.102510  0.097800  0.120103
4    5  interday  15      e       relu  0.103635  0.092588  0.119229
y
  size      mode bar market activation     train     valid      test
0    1  intraday   2      e       relu  0.097292  0.099797  0.097772
1    2  intraday   2      e       relu  0.100840  0.102344  0.102600
2    3  intraday   2      e       relu  0.100264  0.110733  0.111958
3    4  intraday   2      e       relu  0.097413  0.098850  0.098556
4    5  intraday   2      e       relu  0.010558  0.009422  0.011884
cnn
  size      mode bar market activation     train     valid      test
0    1  interday  15      e       relu  0.073136  0.084094  0.105808
1    2  interday  15      

In [19]:
import numpy as np

In [7]:
a = np.array([1,2,4, np.nan, 4])

In [13]:
judge = a>3
judge

/Users/geek/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in greater
  """Entry point for launching an IPython kernel.


array([False, False,  True, False,  True])

In [14]:
b = np.array([1,1,2,np.nan,np.nan])

In [15]:
judge*b

array([ 0.,  0.,  2., nan, nan])

In [20]:
a = np.array([1,2,3,4,5,6])

In [22]:
np.sum(a<=4)

4